In [1]:
import pickle
import pandas as pd

from utils import *
config = load_config()

In [2]:
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

front_data = pd.read_pickle(DATA_PATH.joinpath("front_jargons.pkl"))
back_data = pd.read_pickle(DATA_PATH.joinpath("back_jargons.pkl"))
discharge_dataset = pd.read_pickle(DATA_PATH.joinpath('mimic_discharge.pkl'))

In [3]:
len(front_data)

10001

In [4]:
len(back_data)

10001

In [5]:
# filter the overlaping terms

def filter_the_overlapping_terms(front, back) :
    common = front & back
    return common

commons = []
for front, back in zip(front_data, back_data) :
    common = filter_the_overlapping_terms(front, back)
    commons.append(common)


In [10]:
discharge_dataset = discharge_dataset.iloc[:10001,].copy()

In [13]:
discharge_dataset["keywords"] = commons

In [17]:
discharge_dataset.head(3)

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,front_text,discharge_instructions,keywords
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...,\nName: ___ Unit No: _...,"Discharge Instructions:\nDear Ms. ___,\nIt was...","{liver, pain, Discharge, EGD, care, lasix, exc..."
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...,\nName: ___ Unit No: _...,"Discharge Instructions:\nDear ___,\n\n___ was ...","{abdominal, ascites, potassium, fever, abdomin..."
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...,\nName: ___ Unit No: _...,"Discharge Instructions:\nDear Ms. ___,\n\nYou ...","{blood pressure, improved, confusion, low, inf..."


In [36]:
def parse_dataset_into_json(row) :
    text = row['front_text']
    keyword = list(row['keywords'])
    output = ""
    for idx, keys in enumerate(keyword) :
        output += f"- {keys}\n"

    with open(PROJECT_PATH.joinpath("prompts/finetune_instruction.txt")) as f :
        instruction = f.read()

    formated_terms = {"input" : text,
                    "output" : output, 
                    "instruction" : instruction}

    return formated_terms

In [37]:
parsed_dataset = []
for idx, row in discharge_dataset.iterrows() : 
    output = parse_dataset_into_json(row)
    parsed_dataset.append(output)

In [45]:
parsed_dataset[:2]

[{'input': ' \nName:  ___                     Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \nChief Complaint:\nWorsening ABD distension and pain \n \nMajor Surgical or Invasive Procedure:\nParacentesis\n\n \nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago

In [44]:
import json
with open(DATA_PATH.joinpath("discharge_dataset.json"), "w") as f : 
    json.dump(parsed_dataset, f)

In [43]:

discharge_dataset.to_pickle(DATA_PATH.joinpath('discharge_dataset.pkl'))

In [46]:
with open(DATA_PATH.joinpath("discharge_dataset.json"), "r") as f :
    a = json.load(f)